# Compare hologram data with FGCM and MERRA2 on PWV

- author Sylvie Dagoret-Campagne
- creation date : 2025-12-05 : version v11 with ptc
- affiliation : IJCLab
- Kernel @usdf **w_2025_36*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show correlation holo /Merra

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_HoloFGCM01_HoloFGCMMerra2PWV"
prefix = "pwv01"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')


plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import seaborn as sns

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c
from astropy.table import Table
from astropy.table import join
from astropy.time import Time


#from astropy.coordinates.earth import EarthLocation, EarthLocation, AltAz, get_sun
from astropy.coordinates import EarthLocation, AltAz, get_sun



from datetime import datetime, timedelta, timezone

from pytz import timezone as  timezone_pytz

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict


plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
from PWV00_parameters import *

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
# Palette par filtre
default_filter_colors = {
    "u_24": "tab:blue",
    "g_6":  "tab:green",
    "r_57": "tab:red",
    "i_39": "tab:orange",
    "z_20": "tab:gray",
    "y_10": "black"
}

In [ ]:
default_fhcolors = {
    'empty': "blue", 
    'FELH0600': "purple",
    'OG550_65mm_1':"red",
}

In [ ]:
def plot_fgcm_atm_parameter(t, param="pwv", filter_colors=None):
    """
    Trace un paramètre atmosphérique par date, par filtre,
    avec bandes grises = nuit astronomique au site Rubin-LSST.

    Paramètres
    ----------
    t : astropy.Table
        Table jointe avec colonnes 'mjd', 'physicalFilter' et le paramètre choisi
    param : str
        Nom du paramètre à tracer ('pwv', 'o3', 'tau', etc.)
    filter_colors : dict, optional
        Dictionnaire {filter_name: couleur}, sinon palette par défaut
    """
    if filter_colors is None:
        filter_colors = default_filter_colors

    mjd = t['mjd']
    filters = t['physicalFilter']
    values = t[param]

    mask_valid = np.isfinite(values)
    dates_utc = Time(mjd, format='mjd').to_datetime()

    filter_order = list(filter_colors.keys())

    plt.figure(figsize=(18,8))

    # Scatter par filtre
    for f in filter_order:
        m = (filters == f) & mask_valid
        if np.sum(m) > 0:
            plt.scatter(dates_utc[m], values[m], s=12, alpha=0.6,
                        color=filter_colors[f], label=f)

    # Fonctions auxiliaires
    def night_astronomical_utc(day_mjd, location):
        t_start = Time(day_mjd, format='mjd')
        t_grid = t_start + np.arange(0, 1.5, 5/1440)  # 1.5 jour pour capturer la nuit complète
        altaz = AltAz(obstime=t_grid, location=location)
        sun_alt = get_sun(t_grid).transform_to(altaz).alt
        mask_night = sun_alt < -18*u.deg
        night_times = t_grid[mask_night]
        if len(night_times) == 0:
            return None, None
        return night_times[0], night_times[-1]

    # Bandes grises = nuit astronomique
    start_day = int(np.floor(mjd.min()))
    end_day   = int(np.ceil(mjd.max()))
    all_days = np.arange(start_day, end_day, 1)

    for day_mjd in all_days:
        start_night, end_night = night_astronomical_utc(day_mjd, lsst)
        if start_night is not None:
            plt.axvspan(start_night.datetime, end_night.datetime,
                        color='gray', alpha=0.05)

    # Format axe X
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d:%H'))
    # Locator pour avoir au moins 1 tick par semaine
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO, interval=1))
    #ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
    ax.xaxis.set_major_locator(mdates.AutoDateLocator(minticks=7, maxticks=15))
    # Tick mineur : 1 par jour
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    
    plt.xticks(rotation=45)

    plt.xlabel("Date (UTC)")
    plt.ylabel(f"{param.upper()}")
    plt.title(f"{param.upper()} vs Date (colored by filter)\nGray = astronomical night LSST")
    plt.legend(title="Filter", markerscale=1.5)
    plt.grid(True, alpha=0.3)
    plt.suptitle(suptitle)
    plt.tight_layout()
    figname =f"{pathfigs}/{prefix}_fgcm_{param}"+figtype
    plt.savefig(figname)
    plt.show()

In [ ]:
def night_astronomical_utc(day_mjd, location):
    """Calcule le début et la fin de la nuit astronomique pour un MJD donné."""
    t_start = Time(day_mjd, format='mjd')
    t_grid = t_start + np.arange(0, 1.5, 5/1440)  # 1.5 jour pour capturer la nuit complète
    altaz = AltAz(obstime=t_grid, location=location)
    sun_alt = get_sun(t_grid).transform_to(altaz).alt
    mask_night = sun_alt < -18*u.deg
    night_times = t_grid[mask_night]
    if len(night_times) == 0:
        return None, None
    return night_times[0], night_times[-1]



def plot_holofgcmmerra2_atm_parameter(
    df_h, df_m, t_fgcm,
    param_h="PWV [mm]_x", param_eh="PWV [mm]_err_x", param_grouph="FILTER",
    param_m2="TQV", param_fgcm="pwv", YMIN=0., YMAX=15.,
    filter_colors=None, fhcolors=None, lsst=None, suptitle="", pathfigs=".", prefix="fig", figtype=".png"
):
    """
    Trace un paramètre atmosphérique par date et par filtre, avec bandes grises pour la nuit astronomique.
    """
    # --- Initialisation ---
    if filter_colors is None:
        filter_colors = default_filter_colors  # À définir selon tes besoins
    if fhcolors is None:
        fhcolors = filter_colors  # Par défaut, même palette

    # --- Préparation des données ---

    # FGCM
    mjd2 = t_fgcm['mjd']
    filters2 = t_fgcm['physicalFilter']
    values2 = t_fgcm[param_fgcm]
    mask_valid2 = np.isfinite(values2)
    dates_utc2 = Time(mjd2, format='mjd').to_datetime()

    # MERRA2
    mjd1 = df_m['mjd'].values
    values1 = df_m[param_m2].values
    dates_utc1 = Time(mjd1, format='mjd').to_datetime()

    # Auxtel Holo
    dates_utc0 = df_h["Time"].values

    
    # --- Calcul des limites en X ---
    union_dates = pd.concat([
        pd.Series(dates_utc0),
        pd.Series(dates_utc2)
    ])
    xmin, xmax = union_dates.min(), union_dates.max()

    # --- Création de la figure ---
    plt.figure(figsize=(20, 8))
    ax = plt.gca()

    # --- Tracé des données ---
    # Hologram
    for filt, group in df_h.groupby(param_grouph):
        ax.errorbar(
            group["Time"], group[param_h], yerr=group[param_eh],
            fmt="+", label=f"holo : {filt}",
            color=fhcolors[filt], ecolor=fhcolors[filt],
            elinewidth=1.5, capsize=2, markersize=4
        )

    # FGCM
    for f in filter_colors:
        m = (filters2 == f) & mask_valid2
        if np.sum(m) > 0:
            ax.scatter(
                dates_utc2[m], values2[m], s=30, alpha=0.6,
                color=filter_colors[f], label=f"fgcm : {f}"
            )

    # Merra2
    ax.plot(dates_utc1, values1, marker='.', lw=0.5, ms=1, color="darkblue", label="Merra2")

    
    # --- Nuits astronomiques ---
    mjd = np.union1d(df_h["mjd"].values, mjd2)
    start_day, end_day = int(np.floor(mjd.min())), int(np.ceil(mjd.max()))
    for day_mjd in range(start_day, end_day + 1):
        start_night, end_night = night_astronomical_utc(day_mjd, lsst)
        if start_night is not None:
            ax.axvspan(start_night.datetime, end_night.datetime, color='gray', alpha=0.05)

    # --- Format de l'axe X ---
    ax.set_xlim(xmin, xmax)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H'))
    ax.xaxis.set_major_locator(mdates.AutoDateLocator(minticks=7, maxticks=15))
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    plt.xticks(rotation=45)

    # --- Légende optimisée ---
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # Supprime les doublons
    ax.legend(
        by_label.values(), by_label.keys(),
        title="Filters", bbox_to_anchor=(1.01, 1), loc='upper left',
        borderaxespad=0., frameon=True, fontsize='small'
    )

    # --- Finalisation ---
    ax.set_ylim(YMIN, YMAX)
    ax.set_xlabel("Date (UTC)")
    ax.set_ylabel(f"{param_fgcm.upper()}")
    ax.set_title(f"{param_fgcm.upper()} vs Date (colored by filter)\nGray = astronomical night LSST")
    ax.grid(True, alpha=0.3)
    plt.suptitle(suptitle, y=0.98)
    plt.tight_layout(rect=[0, 0, 0.95, 0.95])  # Ajuste l'espace pour la légende

    # --- Sauvegarde ---
    figname = f"{pathfigs}/{prefix}_holofgcmmerra2_{param_fgcm}{figtype}"
    plt.savefig(figname, bbox_inches='tight')
    plt.show()



## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone_pytz('America/Santiago')

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

### Spectro Hologram data

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## FGCM

In [ ]:
# Rubin-LSST / Cerro Pachón
lsst = EarthLocation(lat=-30.2417*u.deg, lon=-70.7366*u.deg, height=2663*u.m)

In [ ]:
REPO_URI  = "dp2_prep"
#collection = "LSSTCam/runs/DRP/20250417_20250723/d_2025_11_21/DM-53374"
collection = "LSSTCam/runs/DRP/20250417_20250921/w_2025_49/DM-53545" 
strcollection = collection.replace("/","_")
strrepo = REPO_URI.replace("/","_")
suptitle = f"fgcm = repo {REPO_URI}, coll = {collection}"

In [ ]:
#fgcm_filename = "../../2025-12-04-FGCM/data/fgcm_rdp2_prep_cLSSTCam_runs_DRP_20250417_20250723_d_2025_11_21_DM-53374_20251205_095454.fits"
fgcm_filename = "../../2025-12-04-FGCM/data/fgcm_rdp2_prep_cLSSTCam_runs_DRP_20250417_20250921_w_2025_49_DM-53545_20251212_122849.fits"
t_fgcm = Table.read(fgcm_filename)
#df_fgcm = t_fgcm.to_pandas()

In [ ]:
plot_fgcm_atm_parameter(t_fgcm)

### MERRA2

- filename is defined on `PWV00_parameters.py`

In [ ]:
df_m = pd.read_csv(filename_m2,index_col=0)
# Must convert the string Time into _datetime to be plotted 
# 1. Convertir en datetime si ce n'est pas déjà fait
df_m["Time"] = pd.to_datetime(df_m["Time"])
# 2. Convertir en MJD via astropy
# On passe par une liste de strings ISO
time_iso_strings = df_m['Time'].dt.strftime('%Y-%m-%dT%H:%M:%S').tolist()
df_m['mjd'] = Time(time_iso_strings, format='isot').mjd

## Initialisation

### Read the file
- `atmfilename` is defined in `PW00_parameters.py` 

In [ ]:
#version_run = "run_v7"
version_run = "run_v12"
FLAG_REPO_EMBARGO = map_run_butler_embargo[version_run]

In [ ]:
atmfilename = mergedextractedfilesdict[version_run]
tag = legendtag[version_run] 

In [ ]:
the_collection = butlerusercollectiondict[version_run] 
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
time_iso_strings = df_spec['Time'].dt.strftime('%Y-%m-%dT%H:%M:%S').tolist()
df_spec['mjd'] = Time(time_iso_strings, format='isot').mjd

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec[["id","FILTER"]]

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
print(list(df_spec.columns))

## Suppress Blue filters

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]

In [ ]:
# Compter le nombre d’entrées par nightObs et FILTER
counts = df_spec.groupby(["nightObs", "FILTER"]).size().unstack(fill_value=0)

# Plot en barres empilées
counts.plot(kind="bar", stacked=False, figsize=(18,6))

plt.ylabel("Nombre d'entrées")
plt.xlabel("nightObs")
plt.title(f"Nombre d'entrées par FILTER et par nightObs, {tag}")
plt.legend(title="FILTER")
plt.tight_layout()
plt.show()


### Target used

In [ ]:
df_spec["TARGET"].unique()

### Define if a target is faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:10]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

## Select with collimator

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]

### Apply or not correction on errors related to PWV repeatability

In [ ]:
# Take into account Photometric Repeatability
if FLAG_CORRECTFOR_PWV_REPEAT:
    if FLAG_CORRECTFOR_PWV_REPEAT_RATIO:
        df_spec["PWV [mm]_err_x"] =  df_spec["PWV [mm]_err_x"] * FACTORERR_PWV_REPEAT
    else:
        df_spec["PWV [mm]_err_x"] =  np.sqrt(df_spec["PWV [mm]_err_x"]**2  +   SIGMA_PWV_REPEAT**2)
    

## Apply Quality selection

### Compute relative time to Mid-night

In [ ]:
def GetTimeToMidNight(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(local_time.year,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
    local_time_midnight = datetime(local_time_new.year,local_time_new.month,local_time_new.day)
    dt_hour = (local_time_new -local_time_midnight).seconds/3600.

    # we took the previous night mid-night , must subtract 24H
    if dt_hour > 12.:
        dt_hour_new = (dt_hour - 24.)
    else:
        dt_hour_new = dt_hour
        
    return dt_hour_new

In [ ]:
df_spec["dt_midnight"] = df_spec.apply(GetTimeToMidNight,axis=1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
df_spec["dt_midnight"].hist(bins=48,range=(-12,12),ax=ax,facecolor="blue") 
ax.set_xlabel("time relative to midnight (hour)")
ax.set_title("Observation time")

### Compute Date relative to January

In [ ]:
def GetDateToMidJanuary(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new)

In [ ]:
#df_spec["Time_january"] = df_spec.apply(GetDateToMidJanuary,axis=1)

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    observing_time = Time(row['DATE-OBS'], scale='utc', location=observing_location)

    # time at the location , either before or after midnight
    local_time =  observing_time.to_datetime(timezone=tz)

    # take time independent  of any location now
    local_time_new = datetime(2024,local_time.month,local_time.day,local_time.hour,local_time.minute,local_time.second)
           
    return pd.to_datetime(local_time_new),local_time.year 

In [ ]:
def GetDateToMidJanuaryAndYear(row):
    # DATE-OBS is already UTC in FITS; no need for a location
    t = Time(row['DATE-OBS'], scale='utc')
    utc_time = t.to_datetime(timezone=timezone.utc)

    # keep month/day/hour/minute/second in UTC, but set year=2024
    aligned = utc_time.replace(year=2024)

    return pd.to_datetime(aligned), utc_time.year

In [ ]:
def date_to_aligned_year(row):
    t = Time(row["DATE-OBS"], scale="utc", location=observing_location)
    local = t.to_datetime(timezone=tz)
    return local.replace(year=2024), local.year

In [ ]:
def date_to_aligned_year(row):
    t = Time(row["DATE-OBS"], scale="utc", location=observing_location)
    local = t.to_datetime(timezone=tz)

    aligned = local.replace(year=2024)  # preserves month/day/hour/min/sec
    return pd.Timestamp(aligned), local.year

In [ ]:
df_spec[["Time_january","Year"]] = df_spec.apply(GetDateToMidJanuaryAndYear,axis=1,result_type="expand")

In [ ]:
df_spec[["Time_january","Time","Year"]]

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Compute night midnights

In [ ]:
def GetNightMidnightsDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of midnights
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        #sel_flag = df_spec["nightObs"]== nightobs
        #df_night = df_spec[sel_flag]
        #tmin = df_night["Time"].min()-Dt
        #tmax = df_night["Time"].max()+Dt
        nightstr = datetime.strptime(str(nightobs), "%Y%m%d")
        midnight = get_astronomical_midnight(site_lsst, nightstr.date())
        d[nightobs] = midnight
        
    return d

In [ ]:
dnidnights = GetNightMidnightsDict(df_spec)

## Plot all data

In [ ]:
cut  = ~(df_spec["OUTPRESS"] == 743.00) 

In [ ]:
TMIN

In [ ]:
type(datetime_WITHCOLLIMATOR)

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
#df_m.plot(x="Time",y="PShP",ax=ax,marker=".",c="b",lw=0.5,label="Merra2",ms=1,legend=leg)   
df_spec.plot(x="Time",y="OUTPRESS",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("OUTPRESS")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Pressure (before quality cuts) {tag}")
ax.legend()

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)


#ax.set_ylim(0.,15.)

figname =f"{pathfigs}/{prefix}pressure_allpoints_allnights_nocuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")


date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(18,8))
ax  = axs
leg=ax.get_legend()

ax.set_xlim(TMIN,TMAX) 
df_m.plot(x="Time",y="TQV",ax=ax,marker=".",c="g",lw=0.5,label="Merra2",ms=1,legend=leg)   

#df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)


for filt, group in df_spec.groupby("FILTER"):
    ax.errorbar(
                    group["Time"],
                    group["PWV [mm]_x"],
                    yerr= group["PWV [mm]_err_x"],
                    fmt="o",
                    label=filt,
                    color=default_fhcolors[filt],
                    ecolor=default_fhcolors[filt],
                    elinewidth=1.5,
                    capsize=2,
                    markersize=5
                    )


ax.set_ylabel("PWV [mm]_x")
ax.tick_params(axis="x", rotation=45)
ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo (before cuts, {tag})")
ax.legend(loc="upper right")

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)


for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5)

ax.set_ylim(PWVMIN,PWVMAX)
ax.grid()

figname =f"{pathfigs}/{prefix}_pwv_allpoints_allnights_withmerra2_noqualcuts"+figtype
plt.savefig(figname)
plt.show()


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
df_spec[cut].plot.scatter(y="PWV [mm]_x",x="OUTPRESS",marker="+",ax=ax)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
df_spec[cut].plot.scatter(y="P [hPa]",x="OUTPRESS",ax=ax,alpha=0.3)
ax.set_ylim(500,700)
#ax.set_ylim(0,700)

## Apply Quality selection cuts

In [ ]:
cut = getSelectionCut(df_spec) 
cut_nopolar = getSelectionCutNoPolar(df_spec) 
cut_nopolar_bright = getSelectionCutNoPolar(df_spec) & (~df_spec["isFaint"])
cut_nopolar_faint = getSelectionCutNoPolar(df_spec) & (df_spec["isFaint"])
cut_wthpolar = getSelectionCutWithPolar(df_spec)

In [ ]:
df_spec_sel = df_spec[cut]
df_spec_np = df_spec[cut_nopolar] 
df_spec_np_b = df_spec[cut_nopolar_bright]
df_spec_np_f = df_spec[cut_nopolar_faint]
df_spec_wp = df_spec[cut_wthpolar]

In [ ]:
print("Total number of Spectra          : ",len(df_spec))
print("Number of selected Spectra       : ",len(df_spec_sel))
print("Number of selected Polars        : ",len(df_spec_wp))
print("Number of selected Non-Polars    : ",len(df_spec_np))
print("Number of selected Non-Polars Bright : ",len(df_spec_np_b))
print("Number of selected Non-Polars Faint  : ",len(df_spec_np_f))

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)
df_spec_np.reset_index(drop=True,inplace=True)
df_spec_wp.reset_index(drop=True,inplace=True) 
df_spec_np_b.reset_index(drop=True,inplace=True)
df_spec_np_f.reset_index(drop=True,inplace=True)

In [ ]:
#List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
print("Polar            :",len(df_spec_wp["TARGET"].unique()),"\t", df_spec_wp["TARGET"].unique()) 
print("Non Polar        :",len(df_spec_np["TARGET"].unique()),"\t" ,df_spec_np["TARGET"].unique())
print("Non Polar Bright :",len(df_spec_np_b["TARGET"].unique()),"\t" ,df_spec_np_b["TARGET"].unique())
print("Non Polar Faint  :",len(df_spec_np_f["TARGET"].unique()),"\t",df_spec_np_f["TARGET"].unique())

## Plot all data

### Plot PWV

In [ ]:
plot_holofgcmmerra2_atm_parameter(
    df_spec, df_m, t_fgcm,
    param_h="PWV [mm]_x", param_eh="PWV [mm]_err_x",
    param_grouph="FILTER", param_m2="TQV", param_fgcm="pwv",
    YMIN=0., YMAX=15.,
    filter_colors = default_filter_colors, fhcolors = default_fhcolors,
    lsst=lsst, suptitle = suptitle, pathfigs = pathfigs, prefix = prefix
)

### Plot Ozone

In [ ]:
plot_fgcm_atm_parameter(t_fgcm, param="o3")

In [ ]:
df_spec_o3 = df_spec[df_spec["FILTER"].isin(["empty"]) ]

In [ ]:
plot_holofgcmmerra2_atm_parameter(
    df_spec_o3, df_m, t_fgcm,
    param_h="ozone [db]_x", param_eh="ozone [db]_err_x",
    param_grouph="FILTER", param_m2="TO3", param_fgcm="o3",
    YMIN=0., YMAX=600.,
    filter_colors = default_filter_colors, fhcolors = default_fhcolors,
    lsst=lsst, suptitle = suptitle, pathfigs = pathfigs, prefix = prefix
)

In [ ]:
df_spec

## Find which data are in between FGCM dates

In [ ]:
DATEMIN = "2025-04-17"
DATEMAX = "2025-09-21"
DATEMIN = pd.to_datetime(DATEMIN).tz_localize("UTC")
DATEMAX = pd.to_datetime(DATEMAX).tz_localize("UTC")

In [ ]:
# Sélection par intervalle
df_filtered = df_spec[(df_spec["DATE-OBS"] >= DATEMIN) & 
                      (df_spec["DATE-OBS"] <= DATEMAX)]

In [ ]:
df_filtered = df_filtered.reset_index(drop=True)

In [ ]:
df_filtered

In [ ]:
df_filtered["id"]